Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [2]:
display(melb_data['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Приводим признак даты в формат datetime. Используем функции pd.to_datetime(). 

In [1]:
melb_data['Date'] = pd.to_datetime(melb_data['Date'], dayfirst=True)
display(melb_data['Date'])

NameError: name 'pd' is not defined

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64. Рассмотрим несколько возможностей этого типа данных.

                    ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [6]:
year_sold = melb_data['Date'].dt.year
print(year_sold)
print('Min year sold:', year_sold.min())
print('Max year sold:', year_sold.max())
print('Mode year sold:', year_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold: 2016
Max year sold: 2017
Mode year sold: 0    2017
Name: Date, dtype: int32


В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.

Из результатов видно, что данные представлены за интервал с 2016 по 2017 год и наибольшее количество объектов было продано в 2017 году.

Примечание. Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [9]:
melb_data['MonthSale'] = melb_data['Date'].dt.month
melb_data['MonthSale'].value_counts(normalize=True)

MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно).

                РАБОТА С ИНТЕРВАЛАМИ

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [10]:
delta_days = melb_data['Date'] - pd.to_datetime('2016-01-01')
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [11]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [12]:
melb_data['AgeBuilding'] = melb_data['Date'].dt.year - melb_data['YearBuilt']
display(melb_data['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [13]:
melb_data = melb_data.drop('YearBuilt', axis=1)

✍ Мы рассмотрели основные концепции работы с временными признаками в DataFrame.

Временные признаки часто становятся важной частью анализа данных и построения моделей, особенно когда тематика связана со сферой, в которой присутствует непостоянство во времени (зависимость от месяца, дня недели, времени суток), например сфера продаж и услуг. Важно уметь работать с такими признаками, чтобы извлекать из представленных данных максимум полезной информации и строить качественные прогнозы.

А теперь предлагаем вам закрепить пройденный материал ↓

* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

Задание 3.3

Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.


In [19]:
melb_data['WeekdaySale'] = melb_data['Date'].dt.day_of_week
weekend_count = melb_data[(melb_data['WeekdaySale'] == 5) | (melb_data['WeekdaySale'] == 6)].shape[0]
print(weekend_count)
display(melb_data)

12822


,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,MonthSale,AgeBuilding,WeekdaySale
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,...,126.0,Yarra,-37.79960,144.99840,Northern Metropolitan,4019,"-37.7996, 144.9984",12,46,5
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,...,79.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019,"-37.8079, 144.9934",2,116,3
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,...,150.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019,"-37.8093, 144.9944",3,117,5
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,2017-03-04,2.5,...,126.0,Yarra,-37.79690,144.99690,Northern Metropolitan,4019,"-37.7969, 144.9969",3,47,5
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2016-06-04,2.5,...,142.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019,"-37.8072, 144.9941",6,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,2017-08-26,16.7,...,126.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392,"-37.90562, 145.16761",8,36,5
13576,13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,2017-08-26,6.8,...,133.0,NaN,-37.85927,144.87904,Western Metropolitan,6380,"-37.85927, 144.87904",8,22,5
13577,13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,2017-08-26,6.8,...,126.0,NaN,-37.85274,144.88738,Western Metropolitan,6380,"-37.85274, 144.88738",8,20,5
13578,13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,2017-08-26,6.8,...,157.0,NaN,-37.85908,144.89299,Western Metropolitan,6380,"-37.85908, 144.89299",8,97,5


Задание 3.4

В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?

In [21]:
df = pd.read_csv('http://bit.ly/uforeports')
display(df)
df['Time'] = pd.to_datetime(df.Time)
print(df['Time'].dt.year.mode()[0])

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


1999


Задание 3.5

Найдите средний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV).

In [25]:
df['Date'] = df['Time'].dt.date
print(df[df['State']=='NV']['Date'].diff().dt.days.mean())
display(df)


68.92932862190813


,City,Colors Reported,Shape Reported,State,Time,Date
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00,1930-06-01
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00,1930-06-30
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00,1931-02-15
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00,1931-06-01
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00,1933-04-18
...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,2000-12-31 23:00:00,2000-12-31
18237,Spirit Lake,NaN,DISK,IA,2000-12-31 23:00:00,2000-12-31
18238,Eagle River,NaN,NaN,WI,2000-12-31 23:45:00,2000-12-31
18239,Eagle River,RED,LIGHT,WI,2000-12-31 23:45:00,2000-12-31
